### Note
From this repo, OpenAI compatibility only, Google GenAI is stupid

### Why?
Text chunking is an essential step in Retrieval-Augmented Generation (RAG), where large text bodies are divided into meaningful segments to improve retrieval accuracy. Unlike fixed-length chunking, semantic chunking splits text based on the content similarity between sentences.

### Breakpoint Methods
- **Percentile**: Find the Xth percentile of all similarity differences and split chunks where drop is greater than this value
- **Standard Deviation**: Split where similarity drops more than X standard deviations below them
- **Interquartile Range(IQR)**: Use the interquartile distance (Q3 - Q1) to determine split points

### 1. Set up env

In [1]:
import fitz
from dotenv import load_dotenv
import os
import numpy as np
import json
import time
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F
from openai import OpenAI
from sklearn.metrics.pairwise import cosine_similarity

### 2. Extract text

In [2]:
def extract_text_from_pdf(pdf_path):
    """
    Extracts text from a PDF file and prints the first `num_chars` characters.

    Args:
    pdf_path (str): Path to the PDF file.

    Returns:
    str: Extracted text from the PDF.
    """
    # Open the PDF file
    mypdf = fitz.open(pdf_path)
    all_text = ""  # Initialize an empty string to store the extracted text

    # Iterate through each page in the PDF
    for page in mypdf:
        all_text += page.get_text("text") + " "

    return all_text.strip()  # Return the extracted text
# Define the path to the PDF file
pdf_path = "data/AI_Information.pdf"

# Extract text from the PDF file
extracted_text = extract_text_from_pdf(pdf_path)

# Print the first 500 characters of the extracted text
print(extracted_text[:500])

Understanding Artificial Intelligence 
Chapter 1: Introduction to Artificial Intelligence 
Artificial intelligence (AI) refers to the ability of a digital computer or computer-controlled robot 
to perform tasks commonly associated with intelligent beings. The term is frequently applied to 
the project of developing systems endowed with the intellectual processes characteristic of 
humans, such as the ability to reason, discover meaning, generalize, or learn from past 
experience. Over the past f


### 3. Create sentence-level embedding

We split text into sentences and generate embeddings.

I implement embed function using pretrained model because Gemini limits requests per minute.

This model runs smoothly on my GTX 1650 with 4 VRAM

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AutoModel.from_pretrained("BAAI/bge-base-en-v1.5").to(device)
tokenizer = AutoTokenizer.from_pretrained("BAAI/bge-base-en-v1.5")

def create_embeddings(text):
    """
    Create embeddings for text using the loaded embedding model.

    Args:
        text: str or list of str - input text(s) to embed

    Returns:
        list[np.ndarray]: list of normalized embeddings, each of shape (dim,)
    """
    # Handle single string input
    is_string = isinstance(text, str)
    if is_string: text = [text]

    # Tokenize with error handling
    try:
        inputs = tokenizer(
            text,
            padding=True,
            truncation=True,
            max_length=512,
            return_tensors="pt"
        ).to(device)
    except Exception as e:
        print(f"Tokenization error: {e}")
        return None

    # Generate embeddings with no gradient computation
    try:
        with torch.no_grad():
            output = model(**inputs)
            # Use CLS token embedding [CLS] at position 0
            cls_emb = output.last_hidden_state[:, 0, :]
            # L2 normalize for cosine similarity
            emb_normalized = F.normalize(cls_emb, p=2, dim=1)

        # Convert to list of np.ndarray, each of shape (dim,)
        embeddings = [emb.cpu().numpy() for emb in emb_normalized]

        # Return single embedding if input was single string
        return embeddings

    except Exception as e:
        print(f"Embedding generation error: {e}")
        return None


In [4]:
# Splitting text into sentences
sentences = extracted_text.split(". ")  # Optional: dùng nltk/sentencepiece cho chính xác hơn

# Get all embeddings
embeddings = create_embeddings(sentences)

print(f"Generated {len(embeddings)} sentence embeddings.")

Generated 257 sentence embeddings.


In [5]:
embeddings[0].shape

(768,)

### 4. Calculate similarities
We compute cosine similarity between consecutive sentences.

In [6]:
similarities = cosine_similarity(embeddings)
similarities = [similarities[i, i+1] for i in range(len(similarities)-1)]
similarities

[np.float32(0.580174),
 np.float32(0.574335),
 np.float32(0.6697865),
 np.float32(0.70131475),
 np.float32(0.72556984),
 np.float32(0.70602334),
 np.float32(0.60069823),
 np.float32(0.61859655),
 np.float32(0.71721214),
 np.float32(0.8010489),
 np.float32(0.72708505),
 np.float32(0.52289814),
 np.float32(0.41273546),
 np.float32(0.71727836),
 np.float32(0.68105084),
 np.float32(0.7874247),
 np.float32(0.57496053),
 np.float32(0.599665),
 np.float32(0.6533514),
 np.float32(0.49161977),
 np.float32(0.7773316),
 np.float32(0.61359),
 np.float32(0.5431948),
 np.float32(0.59361035),
 np.float32(0.6017686),
 np.float32(0.72643256),
 np.float32(0.7455643),
 np.float32(0.76962733),
 np.float32(0.62178934),
 np.float32(0.6510647),
 np.float32(0.6025127),
 np.float32(0.7794478),
 np.float32(0.5872281),
 np.float32(0.69420683),
 np.float32(0.7226752),
 np.float32(0.60671353),
 np.float32(0.7308327),
 np.float32(0.78676057),
 np.float32(0.63915783),
 np.float32(0.7867347),
 np.float32(0.6032759),


### 5. Implementing Semantic Chunking
implement three different methods for finding breakpoints.

In [7]:
def compute_breakpoints(similarities, method="percentile", threshold=90):
    """
    Computes chunking breakpoints based on similarity drops.

    Args:
    similarities (List[float]): List of similarity scores between sentences.
    method (str): 'percentile', 'standard_deviation', or 'interquartile'.
    threshold (float): Threshold value (percentile for 'percentile', std devs for 'standard_deviation').

    Returns:
    List[int]: Indices where chunk splits should occur.
    """
    # Determine the threshold value based on the selected method
    if method == "percentile":
        # Calculate the Xth percentile of the similarity scores
        threshold_value = np.percentile(similarities, threshold)
    elif method == "standard_deviation":
        # Calculate the mean and standard deviation of the similarity scores
        mean = np.mean(similarities)
        std_dev = np.std(similarities)
        # Set the threshold value to mean minus X standard deviations
        threshold_value = mean - (threshold * std_dev)
    elif method == "interquartile":
        # Calculate the first and third quartiles (Q1 and Q3)
        q1, q3 = np.percentile(similarities, [25, 75])
        # Set the threshold value using the IQR rule for outliers
        threshold_value = q1 - 1.5 * (q3 - q1)
    else:
        # Raise an error if an invalid method is provided
        raise ValueError("Invalid method. Choose 'percentile', 'standard_deviation', or 'interquartile'.")

    # Identify indices where similarity drops below the threshold value
    return [i for i, sim in enumerate(similarities) if sim < threshold_value]

# Compute breakpoints using the percentile method with a threshold of 90
breakpoints = compute_breakpoints(similarities, method="percentile", threshold=90)
breakpoints


[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 44,
 45,
 46,
 48,
 49,
 50,
 51,
 52,
 53,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 72,
 73,
 74,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 106,
 107,
 108,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 128,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 152,
 155,
 156,
 158,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 170,
 171,
 172,
 173,
 175,
 176,
 177,
 178,
 180,
 181,
 182,
 183,
 184,
 186,
 188,
 189,
 190,
 191,
 192,
 193,
 194,
 195,
 196,
 198,
 199,
 200,
 201,
 202,
 203,
 204,
 206,


### 6. Splitting Text into Semantic Chunks

We split the text based on computed breakpoints.


In [8]:
def split_into_chunks(sentences, breakpoints):
    """
    Splits sentences into semantic chunks.

    Args:
    sentences (List[str]): List of sentences.
    breakpoints (List[int]): Indices where chunking should occur.

    Returns:
    List[str]: List of text chunks.
    """
    chunks = []  # Initialize an empty list to store the chunks
    start = 0  # Initialize the start index

    # Iterate through each breakpoint to create chunks
    for bp in breakpoints:
        # Append the chunk of sentences from start to the current breakpoint
        chunks.append(". ".join(sentences[start:bp + 1]) + ".")
        start = bp + 1  # Update the start index to the next sentence after the breakpoint

    # Append the remaining sentences as the last chunk
    chunks.append(". ".join(sentences[start:]))
    return chunks  # Return the list of chunks

# Create chunks using the split_into_chunks function
text_chunks = split_into_chunks(sentences, breakpoints)

# Print the number of chunks created
print(f"Number of semantic chunks: {len(text_chunks)}")

# Print the first chunk to verify the result
print("\nFirst text chunk:")
print(text_chunks)


Number of semantic chunks: 231

First text chunk:
['Understanding Artificial Intelligence \nChapter 1: Introduction to Artificial Intelligence \nArtificial intelligence (AI) refers to the ability of a digital computer or computer-controlled robot \nto perform tasks commonly associated with intelligent beings.', 'The term is frequently applied to \nthe project of developing systems endowed with the intellectual processes characteristic of \nhumans, such as the ability to reason, discover meaning, generalize, or learn from past \nexperience.', 'Over the past few decades, advancements in computing power and data availability \nhave significantly accelerated the development and deployment of AI.', '\nHistorical Context \nThe idea of artificial intelligence has existed for centuries, often depicted in myths and fiction.', '\nHowever, the formal field of AI research began in the mid-20th century.', 'The Dartmouth Workshop \nin 1956 is widely considered the birthplace of AI.', 'Early AI resea

### Creating Embeddings for Semantic Chunks

We create embeddings for each chunk for later retrieval.

In [9]:
# Create chunk embeddings using the create_embeddings function
chunk_embeddings = create_embeddings(text_chunks)


In [10]:
# chunk_embedding

### Performing Semantic Search

We implement cosine similarity to retrieve the most relevant chunks.

In [11]:
def semantic_search(query, text_chunks, chunk_embeddings, k=5):
    """
    Finds the most relevant text chunks for a query.

    Args:
        query (str): Search query.
        text_chunks (List[str]): List of text chunks.
        chunk_embeddings (List[np.ndarray]): List of chunk embeddings.
        k (int): Number of top results to return.

    Returns:
        List[str]: Top-k relevant chunks.
    """
    # Embed the query
    query_embedding = create_embeddings(query)[0]

    # Ensure query_embedding has correct shape (1, dim)
    query_embedding = query_embedding.reshape(1, -1)

    # Compute cosine similarities
    similarities = np.array([
        cosine_similarity(query_embedding, emb.reshape(1, -1))[0][0]  # scalar similarity
        for emb in chunk_embeddings
    ])

    # Get top-k indices
    top_indices = np.argsort(similarities)[-k:][::-1]

    # Return top-k text chunks
    return [text_chunks[i] for i in top_indices]


In [12]:
# Load the validation data from a JSON file
with open('data/val.json') as f:
    data = json.load(f)

# Extract the first query from the validation data
query = data[0]['question']

# Get top 2 relevant chunks
top_chunks = semantic_search(query, text_chunks, chunk_embeddings, k=2)

# Print the query
print(f"Query: {query}")

# Print the top 2 most relevant text chunks
for i, chunk in enumerate(top_chunks):
    print(f"Context {i+1}:\n{chunk}\n{'='*40}")


Query: What is 'Explainable AI' and why is it considered important?
Context 1:

Explainable AI (XAI) 
Explainable AI (XAI) aims to make AI systems more transparent and understandable. Research in 
XAI focuses on developing methods for explaining AI decisions, enhancing trust, and improving 
accountability.
Context 2:

Transparency and Explainability 
Transparency and explainability are essential for building trust in AI systems.


### Generating a Response Based on Retrieved Chunks

In [13]:
load_dotenv("conf.env")
client = OpenAI(
    api_key=os.getenv("GEMINI_API_KEY"),
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

In [14]:
# Define the system prompt for the AI assistant
system_prompt = "You are an AI assistant that strictly answers based on the given context. If the answer cannot be derived directly from the provided context, respond with: 'I do not have enough information to answer that.'"

def generate_response(system_prompt, user_message, model="gemini-2.5-flash"):
    """
    Generates a response from the AI model based on the system prompt and user message.

    Args:
    system_prompt (str): The system prompt to guide the AI's behavior.
    user_message (str): The user's message or query.
    model (str): The model to be used for generating the response. Default is "meta-llama/Llama-2-7B-chat-hf".

    Returns:
    dict: The response from the AI model.
    """
    response = client.chat.completions.create(
        model=model,
        temperature=0,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_message}
        ]
    )
    return response

# Create the user prompt based on the top chunks
user_prompt = "\n".join([f"Context {i + 1}:\n{chunk}\n=====================================\n" for i, chunk in enumerate(top_chunks)])
user_prompt = f"{user_prompt}\nQuestion: {query}"

# Generate AI response
ai_response = generate_response(system_prompt, user_prompt)


In [15]:
ai_response_text = ai_response.choices[0].message.content
ai_response_text

'Explainable AI (XAI) aims to make AI systems more transparent and understandable. It is considered important because it enhances trust, improves accountability, and is essential for building trust in AI systems.'

### Evaluation

In [16]:
# Define the system prompt for the evaluation system
evaluate_system_prompt = "You are an intelligent evaluation system tasked with assessing the AI assistant's responses. If the AI assistant's response is very close to the true response, assign a score of 1. If the response is incorrect or unsatisfactory in relation to the true response, assign a score of 0. If the response is partially aligned with the true response, assign a score of 0.5."

# Create the evaluation prompt by combining the user query, AI response, true response, and evaluation system prompt
evaluation_prompt = f"User Query: {query}\nAI Response:\n{ai_response_text}\nTrue Response: {data[0]['ideal_answer']}\n{evaluate_system_prompt}"

# Generate the evaluation response using the evaluation system prompt and evaluation prompt
evaluation_response = generate_response(evaluate_system_prompt, evaluation_prompt)

# Print the evaluation response
print(evaluation_response.choices[0].message.content)

Score: 1


In [ ]:
ai_gen = create_embeddings(ai_response_text)[0].reshape(1, -1) # (1,x)
ideal_re = create_embeddings(data[0]['ideal_answer'])[0].reshape(1, -1) # (1,x)
print(ai_gen.shape, ideal_re.shape)

cosine_similarity(ai_gen, ideal_re)

(1, 768) (1, 768)


array([[0.9888819]], dtype=float32)

: 